In [ ]:
import os
from openai import OpenAI
import rich
import requests
import json
from google.colab import userdata

In [ ]:
api_key = userdata.get('OPENAI_API_KEY')
MODEL = "gpt-4o-mini"

openai = OpenAI(api_key=api_key)

**Calling a function and sending the result back to Chat API and Responses API**

Defining function that fetch current weather from API

In [ ]:
def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    print(f"get_weather function called to get weather for latitude = {latitude}, longitude = {longitude}")
    print(f"And result is  = {data['current']['temperature_2m']}")
    return data['current']['temperature_2m']

# Chat Completion API

https://platform.openai.com/docs/guides/function-calling?api-mode=chat

Defining the structure schema of the function to be passed as a tool in the API.

In [ ]:
def get_weather_function_chat():
    return {
        "type": "function",
        "function": { # This property is removed from responses API
            "name": "get_weather",
            "description": "Get the weather for a location. Call this whenever you need to know the weather, for example when a customer asks 'What's the weather like in this city'",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {
                        "type": "number",
                        "description": "Latitude of the location"
                    },
                    "longitude": {
                        "type": "number",
                        "description": "Longitude of the location"
                    }
                },
                "required": ["latitude", "longitude"],
                "additionalProperties": False
            },
            "strict": True
        }
    }

First Step where model will responed with tool call request

In [ ]:
messages=[
    {"role": "developer", "content": "You are a helpful assistant and provide update on weather in a city."},
    {"role": "user", "content": "What's the weather like in Karachi, Pakistan?"}
    # {"role": "user", "content": "NYC"}
]
tools = [get_weather_function_chat()]
response = openai.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools = tools
)

rich.print(response.choices[0])
print("Finish Reason = ", response.choices[0].finish_reason)
rich.print(response.choices[0].message.tool_calls)


Choice(
    finish_reason='tool_calls',
    index=0,
    logprobs=None,
    message=ChatCompletionMessage(
        content=None,
        refusal=None,
        role='assistant',
        annotations=[],
        audio=None,
        function_call=None,
        tool_calls=[
            ChatCompletionMessageToolCall(
                id='call_QN4XxX4Wyrxt98BrU2v8vN3q',
                function=Function(arguments='{"latitude":24.8607,"longitude":67.0011}', name='get_weather'),
                type='function'
            )
        ]
    )
)

Finish Reason =  tool_calls


[
    ChatCompletionMessageToolCall(
        id='call_QN4XxX4Wyrxt98BrU2v8vN3q',
        function=Function(arguments='{"latitude":24.8607,"longitude":67.0011}', name='get_weather'),
        type='function'
    )
]

Second Step where we are calling the `get_weather` function and sending the response back to Chat API

To send the result of function call we need to send specific format object into Chat API call

```
new_message = {
    "role": "tool",
    "content": <output of function call>,
    "tool_call_id": tool_call.id
}
```

In [ ]:
if response.choices[0].finish_reason == "tool_calls": # Check if finish_reason is tool_calls
    tool_call = response.choices[0].message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    latitude = arguments.get("latitude")
    longitude = arguments.get("longitude")
    # weather = get_weather(latitude, longitude) # Both will work
    weather = get_weather(**arguments)
    new_message = { # Properties of this object will be different in responses API
        "role": "tool",
        "content": json.dumps({"latitude": latitude, "longitude": longitude, "weather": weather}),
        "tool_call_id": tool_call.id
    }
    # Important: we will append the previous message (response.choices[0].message)
    messages.append(response.choices[0].message)
    messages.append(new_message)
    # Calling the Chat API again with all the history messages and the new message
    response2 = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    print("Model Response2 = ",response2.choices[0].message.content)
    # Notice the finish_reason of the response, it value is "stop"
    print("Finish Reason = ",response2.choices[0].finish_reason)

get_weather function called to get weather for latitude = 24.8607, longitude = 67.0011
And result is  = 25.5
Model Response2 =  The current weather in Karachi, Pakistan is 25.5°C. If you need more specific details, feel free to ask!
Finish Reason =  stop


# Responses API

https://platform.openai.com/docs/guides/function-calling?api-mode=responses

Note: Check the difference between json objects for function

In [ ]:
def get_weather_function_response():
    return {
        "type": "function",  # There is no function property in the response API
        "name": "get_weather",
        "description": "Get the weather for a location. Call this whenever you need to know the weather, for example when a customer asks 'What's the weather like in this city'",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {
                    "type": "number",
                    "description": "Latitude of the location"
                },
                "longitude": {
                    "type": "number",
                    "description": "Longitude of the location"
                }
            },
            "required": ["latitude", "longitude"],
            "additionalProperties": False
        },
        "strict": True
    }

### Using old way of sending history messages in every call

First Step where model will responed with tool call request

In [ ]:
messages=[
    {"role": "developer", "content": "You are a helpful assistant and provide update on weather in a city."},
    # {"role": "user", "content": "What's the weather like in Karachi, Pakistan?"}
    # {"role": "user", "content": "NYC"}
     {"role": "user", "content": "Berlin"}
]
tools = [get_weather_function_response()]

response = openai.responses.create(
    model=MODEL,
    input=messages,
    tools = tools
)

print("Status = ",response.status) # Status will not indicate the tool call
print(response.output_text) # Empty
rich.print(response.output)
# rich.print(response)

Status =  completed



[
    ResponseFunctionToolCall(
        arguments='{"latitude":52.52,"longitude":13.405}',
        call_id='call_CqCPpXMNpx9dPr9F4SrgPunj',
        name='get_weather',
        type='function_call',
        id='fc_67e2e975e9cc8192b7bfc5d9afd83cfc03f8040ad152ec7e',
        status='completed'
    )
]

Second Step where we are calling the `get_weather` function and sending the response back to Responses API

To send the result of function call we need to send specific format object into Responses API call

Note: object has different property names.

```
{
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": <output of function call>,
}
```

In [ ]:
if response.output[0].type == "function_call": # Check if output type is function_call
    tool_call = response.output[0]
    arguments = json.loads(tool_call.arguments)
    latitude = arguments.get("latitude")
    longitude = arguments.get("longitude")
    # weather = get_weather(latitude, longitude) # Both will work
    weather = get_weather(**arguments)
    new_message = {
        "type": "function_call_output",
        "call_id": tool_call.call_id,
        # "output": str(weather)
        # Because of json object in output Responses API sometimes does not generate expected output
        "output":  json.dumps({"latitude": latitude, "longitude": longitude, "weather": weather}),
    }
    # Important: we will append the tool call (response.output[0])
    messages.append(response.output[0])
    messages.append(new_message)
    # Calling the Responses API again with all the history messages and the new message
    response2 = openai.responses.create(model=MODEL, input=messages,tools = tools)
    print("Model Response2 = ",response2.output_text)
    print("Status = ",response2.status)
    # rich.print(response2)

get_weather function called to get weather for latitude = 52.52, longitude = 13.405
And result is  = 8.0
Model Response2 =  The weather in Berlin is currently quite cool, with a temperature of around 8°C. If you need more specific details like conditions or forecasts, let me know!
Status =  completed


### Using new way of conversation state by sending perivous reponse id

First Step where model will responed with tool call request

In [ ]:
# This section is same as above

messages=[
    {"role": "developer", "content": "You are a helpful assistant and provide update on weather in a city."},
    # {"role": "user", "content": "What's the weather like in Karachi, Pakistan?"}
    # {"role": "user", "content": "NYC"}
     {"role": "user", "content": "Berline"}
]
tools = [get_weather_function_response()]

response = openai.responses.create(
    model=MODEL,
    input=messages,
    tools = tools
)

print("Status = ",response.status) # Status will not indicate the tool call
print(response.output_text) # Empty
rich.print(response.output)

Status =  completed



[
    ResponseFunctionToolCall(
        arguments='{"latitude":52.520008,"longitude":13.404954}',
        call_id='call_dwHoh7PoVzXP8i2FTUBbQR4o',
        name='get_weather',
        type='function_call',
        id='fc_67e2ea01b6e88192b92725629f0034e30f0dfe54a3d226a1',
        status='completed'
    )
]

Second Step where we are calling the `get_weather` function and sending the response back to Responses API

The only difference in below section is how messages are sent.

In [ ]:
if response.output[0].type == "function_call": # Check if output type is function_call
    tool_call = response.output[0]
    arguments = json.loads(tool_call.arguments)
    latitude = arguments.get("latitude")
    longitude = arguments.get("longitude")
    # weather = get_weather(latitude, longitude) # Both will work
    weather = get_weather(**arguments)
    new_message = {
        "type": "function_call_output",
        "call_id": tool_call.call_id,
        # "output": str(weather)
        # Because of json object in output Responses API sometimes does not generate expected output
        "output":  json.dumps({"latitude": latitude, "longitude": longitude, "weather": weather}),
    }
    # Not needed now, because we are sending the previous response id
    # messages.append(response.output[0])

    # Emptying the messages array because we are sending the previous response id,
    # therefore we don't need to send the previous message
    messages = []
    messages.append(new_message)
    # Note the previous_response_id
    response2 = openai.responses.create(model=MODEL, input=messages,tools = tools, previous_response_id=response.id)
    print("Model Response2 = ",response2.output_text)
    print("Status = ",response2.status)

get_weather function called to get weather for latitude = 52.520008, longitude = 13.404954
And result is  = 8.0
Model Response2 =  The current weather in Berlin is around 8°C. If you need more details or specific weather conditions, feel free to ask!
Status =  completed
